<a href="https://colab.research.google.com/github/yunnsss/Fog_Forecast/blob/main/EDA_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/날씨마루 공모전/train_tt.csv')
test = pd.read_csv('/content/drive/MyDrive/날씨마루 공모전/test_tt.csv')

In [ ]:
# # 열 이름에서 'fog_train.'을 제거하고 뒤에 문자열로 열 이름 지정
# train.columns = train.columns.str.replace('fog_train.', '')
# test.columns =test.columns.str.replace('fog_test.', '')

## datetime 만들기

In [ ]:
# 'time' 열의 이름을 'hour'로 변경
# year 값을 2020, 2021, 2022로 변경
train['year'] = train['year'].replace({'I': 2020, 'J': 2021, 'K': 2022})
test['year'] = test['year'].replace({'L': 2023})

train = train.rename(columns={'time': 'hour'})
test = test.rename(columns={'time': 'hour'})

# 'year', 'month', 'day', 'hour', 'minute' 열을 결합하여 datetime 형식으로 변환
train['datetime'] = pd.to_datetime(train[['year', 'month', 'day', 'hour', 'minute']])
test['datetime'] = pd.to_datetime(test[['year', 'month', 'day', 'hour', 'minute']])

# 결측치 처리: 삭제

In [ ]:
# -90 이하의 값을 NaN으로 변경

#train
cols_to_check = ['ws10_deg', 'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts', 'class']
train[cols_to_check] = train[cols_to_check].mask(train[cols_to_check] <= -90)

#test
test_cols_to_check = ['ws10_deg', 'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts']
test[cols_to_check] = test[cols_to_check].mask(test[test_cols_to_check] <= -90)

In [ ]:
# 결측치가 있는 행 제거하여 원래 데이터셋에 반영
train = train.dropna(subset=cols_to_check)
test = test.dropna(subset=test_cols_to_check)

In [ ]:
# vis1이 음수인 값은 class를 1로 치환
train['vis1'] = np.clip(train['vis1'], 1, 2000)

# 이상치 처리: 평균값 대체

In [ ]:
# 이상치를 찾을 특정 열들
cols_to_check = ['ws10_deg', 'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts']

In [ ]:
# 이상치를 평균값으로 대체하는 함수 정의
 def replace_outliers_with_mean(df, columns):
     for col in columns:
         Q1 = df[col].quantile(0.25)
         Q3 = df[col].quantile(0.75)
         IQR = Q3 - Q1
         lower_bound = Q1 - 1.5 * IQR
         upper_bound = Q3 + 1.5 * IQR

        # 이상치를 평균값으로 대체
         df[col] = np.where((df[col] < lower_bound) | (df[col] > upper_bound), df[col].mean(), df[col])

In [ ]:
# 이상치를 평균값으로 대체하는 함수 호출
replace_outliers_with_mean(train, cols_to_check)

In [ ]:
# 이상치를 찾을 특정 열들
cols_to_check = ['ws10_deg', 'ws10_ms', 'ta', 're', 'hm', 'sun10', 'ts']

# 이상치를 평균값으로 대체하는 함수 호출
replace_outliers_with_mean(test, cols_to_check)

### 파생 변수 생성

In [ ]:
# Magnus-Tetens 공식을 이용하여 노점온도(tb) 계산
a = 17.27
b = 237.7
tb = (b * np.log(train['hm']/100) + (a * train['ta']) / (b + train['ta'])) / (a - np.log(train['hm']/100))

# 기온(ta)과 노점온도(tb) 간의 차이 계산
train['ta_tb_diff'] = abs(train['ta'] - tb)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# Magnus-Tetens 공식을 이용하여 노점온도(tb) 계산
a = 17.27
b = 237.7
tb = (b * np.log(test['hm']/100) + (a * test['ta']) / (b + test['ta'])) / (a - np.log(test['hm']/100))

# 기온(ta)과 노점온도(tb) 간의 차이 계산
test['ta_tb_diff'] = abs(test['ta'] - tb)

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
# 계절성과 고려
def add_seasonal_features(df):
    #df['month'] = df['datetime'].dt.month
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
    df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
    return df

train = add_seasonal_features(train)
test = add_seasonal_features(test)

In [ ]:
# train 데이터프레임에서 'st' 변수 삭제
train = train.drop('st', axis=1)
test = test.drop('st', axis=1)

In [ ]:
#train
# 'stn_id' 별 'ta', 'sun10', 'ts'의 평균 계산
mean_values = train.groupby('stn_id')[['ta', 'sun10', 'ts']].mean()

# 각 'stn_id'에 맞게 'st' 값 계산
mean_values['st'] = mean_values.sum(axis=1)

# 'st' 값을 원래 데이터프레임에 추가
train = train.merge(mean_values[['st']], on='stn_id', how='left')

In [ ]:
# 'stn_id' 별 'ta', 'sun10', 'ts'의 평균 계산
mean_values = test.groupby('stn_id')[['ta', 'sun10', 'ts']].mean()

# 각 'stn_id'에 맞게 'st' 값 계산
mean_values['st'] = mean_values.sum(axis=1)

# 'st' 값을 원래 데이터프레임에 추가
test = test.merge(mean_values[['st']], on='stn_id', how='left')

In [ ]:
## 풍속(풍향cos)
#풍향을 라디안 단위로 변환
ws10_deg_rad = np.deg2rad(train.ws10_deg)

#코사인 변수 생성
ws10_cos = np.cos(ws10_deg_rad)

# 풍속*풍향cos
ws10_new = train.ws10_ms*ws10_cos

# 'ws10_cos' 값을 원래 데이터프레임에 추가
train['ws10_new'] = ws10_new

In [ ]:
## 풍속(풍향cos)
#풍향을 라디안 단위로 변환
ws10_deg_rad = np.deg2rad(test.ws10_deg)

#코사인 변수 생성
ws10_cos = np.cos(ws10_deg_rad)

# 풍속*풍향cos
ws10_new = test.ws10_ms*ws10_cos

# 'ws10_cos' 값을 원래 데이터프레임에 추가
test['ws10_new'] = ws10_new

# 정규화

In [ ]:
#Min-Max 정규화 객체 생성
scaler = MinMaxScaler()

# 선택한 열들에 Min-Max 정규화 적용
train[['ta_tb_diff', 'st']] = scaler.fit_transform(train[['ta_tb_diff', 'st']])

In [ ]:
#Min-Max 정규화 객체 생성
scaler = MinMaxScaler()

# 선택한 열들에 Min-Max 정규화 적용
test[['ta_tb_diff', 'st']] = scaler.fit_transform(test[['ta_tb_diff', 'st']])

In [ ]:
# Min-Max 정규화를 적용할 열들 선택
columns_to_normalize = [ 'ws10_deg', 'ws10_ms', 'ta', 'hm', 'sun10', 'ts']

# Min-Max 정규화 객체 생성
scaler = MinMaxScaler()

# 선택한 열들에 Min-Max 정규화 적용
train[columns_to_normalize] = scaler.fit_transform(train[columns_to_normalize])

In [ ]:
# Min-Max 정규화를 적용할 열들 선택
columns_to_normalize = ['ws10_deg', 'ws10_ms', 'ta', 'hm', 'sun10', 'ts']

# Min-Max 정규화 객체 생성
scaler = MinMaxScaler()

# 선택한 열들에 Min-Max 정규화 적용
test[columns_to_normalize] = scaler.fit_transform(test[columns_to_normalize])